In [82]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
import json
from fastapi import FastAPI, Response
from fastapi.responses import JSONResponse
sns.set()
warnings.filterwarnings('ignore')

In [2]:
movies = pd.read_csv('../final_data/combined_data.csv')

In [3]:
movies.sample(2)

,belongs_to_collection,budget,genres,movie_id,original_language,overview,popularity,production_companies,production_countries,release_date,...,title,vote_average,vote_count,production_countries_code,release_year,release_month,release_day,return,director,actors
39099,NaN,0.0,Drama,327040,fr,"Biarritz. 16 year-old George, the high-school ...",2.793311,B Media Développement|Canal+|Full House|Orange...,France,2013-03-12,...,Bang Gang (A Modern Love Story),4.3,99.0,FR,2013,3,12,0.0,Eva Husson,Marilyn Lima|Daisy Broom|Finnegan Oldfield|Lor...
24948,NaN,0.0,Horror,200044,en,Deranged’ follows four girls who go on a bache...,0.702316,Burn Hand Film Productions|EnMar Productions|T...,United Kingdom,2014-11-07,...,Deranged,4.1,7.0,GB,2014,11,7,0.0,Neil Jones,Marcia Do Vales|Craig Fairbrass|Victoria Broom...


#### Seleccion de las columnas
*No selecciono la columna de overview porque si no mi matriz de similitud se va hacer muy pesada y dificil de procesar*

In [4]:
movies = movies[['title','genres','director','popularity']]
movies.sample(2)

,title,genres,director,popularity
38408,Carry On Cleo,Comedy,Gerald Thomas,6.143986
7048,Where the Boys Are,Comedy|Drama|Romance,Henry Levin,0.804731


In [5]:
movies.genres = movies.genres.str.replace('|',' ')

In [6]:
movies['joined_data'] = movies[['title','genres','director']].astype(str).apply(' '.join, axis=1)
movies.sample()

,title,genres,director,popularity,joined_data
11591,Gray Matters,Comedy Romance,Sue Kramer,2.268351,Gray Matters Comedy Romance Sue Kramer


Cálculo de similitud de puntuación


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.metrics.pairwise import linear_kernel
 
# Elejí usar TfidfVectorizer para la vectorizacion de los datos en vez de CountVectorizer. Si bien este ultimo es mejor por cuestiones de memoria, 
# el TfidfVectorizer brinda mayor precision a la hora de buscar una puntuación de similitud en las cadenas de strings

In [8]:
vectorizer = TfidfVectorizer(stop_words='english') # Elimino las palabras mas comunes del ingles

In [9]:
movies.genres.isnull().sum()

2383

Veo cuales son las peliculas que tiene en el campo de género un valor vacio y ordeno el resultado dependiendo la popularidad de la misma

In [11]:
movies.loc[movies.genres.isnull()].sort_values(by='popularity', ascending=False).head(20)

,title,genres,director,popularity,joined_data
20850,The Scapegoat,NaN,Charles Sturridge,7.618217,The Scapegoat nan Charles Sturridge
7889,Fahrenheit 9/11,NaN,Michael Moore,6.839460,Fahrenheit 9/11 nan Michael Moore
30783,My Date with the President's Daughter,NaN,Alex Zamm,5.019029,My Date with the President's Daughter nan Alex...
44772,Notte prima degli esami - Oggi,NaN,Fausto Brizzi,4.345520,Notte prima degli esami - Oggi nan Fausto Brizzi
25866,Viaggi di nozze,NaN,Carlo Verdone,4.323640,Viaggi di nozze nan Carlo Verdone
37395,La Vacanza,NaN,Tinto Brass,4.158529,La Vacanza nan Tinto Brass
43072,I still hide to smoke,NaN,Rayhana Obermeyer,3.589741,I still hide to smoke nan Rayhana Obermeyer
32019,School of Thieves,NaN,Neri Parenti,3.080456,School of Thieves nan Neri Parenti
32024,Fracchia The Human Beast,NaN,Neri Parenti,3.053669,Fracchia The Human Beast nan Neri Parenti
30589,Koyla,NaN,Rakesh Roshan,3.037553,Koyla nan Rakesh Roshan


Como no son peliculas muy conocidas y tampoco son muchas, decido eliminarlas del dataset. Tomo esta decision porque al momento de recurrir a la función de recomendación de película, el modelo de machine learning debe seleccionar entre las peliculas que tienen el mismo género cinematografico para relacionarlas y en los casos de estas peliculas seria irrelevante ya que no tienen defino un género.

In [12]:
movies = movies.dropna(subset='genres')

In [13]:
movies.title.sample().values

array(['The Great Waltz'], dtype=object)

### Desarrollo de la función

In [89]:
def recomendacion(title):
    """Get the recommendation for a given title"""
    # Obtengo los géneros de la pelicula
    if title not in movies.title.unique():
        data_json = {'mensaje':'Pelicula no encontrada'}
    else:
        # Obtengo los géneros de la pelicula
        movie_genres = movies['genres'].loc[movies['title'] == title]
        #movie_genres = movie_genres.values[pd.isna(movie_genres.values) == False].item()
        movie_genres = sorted(movie_genres, key= lambda x: len(x), reverse=True) # Ordenar los géneros en función del len()


        # Separo los generos en una lista
        #genres = [v.split(' ') for v in movie_genres if isinstance(v,str)]

        # Filtro el dataframe con las peliculas que tengan los mismos géneros que la pelicula de interes
        # y con las peliculas que tengan en su titulo el nombre de la pelicula de interes
        similar_movies = movies.loc[(movies.genres.fillna(' ').str.contains(movie_genres[0])) | (movies.title.str.contains(title))].reset_index(drop=True)
        max_len = 3500
        if len(similar_movies) > max_len:
            similar_movies = similar_movies.sort_values(by='popularity', ascending=False).reset_index(drop=True)[:max_len]

        # Obtengo el indice en el dataframe filtrado de la pelicula de interes
        movie_index = similar_movies.loc[similar_movies['title'] == title].index
        
        if len(movie_index) == 0:
            similar_movies = similar_movies.append(movies.loc[movies.title == title]).reset_index(drop=True)
            movie_index = similar_movies.loc[similar_movies['title'] == title].index

        # Vectorizo los datos
        data_matrix = vectorizer.fit_transform(similar_movies.joined_data)
        # Genero la matriz de puntuación de similitud entre las peliculas
        similarity_matrix = cosine_similarity(data_matrix,data_matrix)

        sim_movies = list(enumerate(similarity_matrix[movie_index[0]]))
        sim_movies = sorted(sim_movies, key=lambda x: x[1], reverse=True)
        top_similar_movies = [similar_movies.loc[i, 'title'] for i, _ in sim_movies[1:20] if similar_movies.loc[i, 'title'] != title][:5]
        data_json = {'similar_movies':top_similar_movies}

    
    return data_json

### Probando la función

In [90]:
recomendacion('The Avengers')

{'similar_movies': ['Avengers: Age of Ultron',
  'The Island',
  'Armageddon',
  'I Am Number Four',
  'Mad Max']}